In [10]:
import twitter
import tweepy
import csv
import time
from datetime import datetime

con_key = 'xxxxxxxxxxxxxxxxxxxxxxxxx'
con_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
acc_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
acc_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

In [11]:
###############################################################################
#
# get_tweepy_api(): Return authenticated twitter service api
#
# NOTE: not using tweepy, use twitter instead
###############################################################################
def get_tweepy_api():
    auth = tweepy.OAuthHandler(con_key, con_secret)
    auth.set_access_token(acc_token, acc_secret)
    tweepy_api= tweepy.API(auth)
   
    return tweepy_api

In [13]:
###############################################################################
#
# get_twitter_api(): Return authenticated twitter service api
#
# NOTE: the only function called from main()
###############################################################################
def get_twitter_api():
    auth = twitter.oauth.OAuth(acc_token, acc_secret, con_key, con_secret)
    twitter_api = twitter.Twitter(auth=auth)

    return twitter_api

In [14]:
###############################################################################
#
# twitter_search(): Search twitter in batches. [main() code modeled from 
# Mining the Social Web, 2nd Ed. by M. Russell below]
#
###############################################################################
def twitter_search(twitter_api, q, max_results=200, **kw):
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    statuses = search_results['statuses']
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    for _ in range(10): # 10*100 = 1000
        try:
            print ("In twitter_search loop")
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

In [15]:
###############################################################################
#
# main(): 
#
###############################################################################
def main():
    twitter_service_api = get_twitter_api()
    f = open("TweetsX3_Data.csv", "a", newline="")
    tweet_writer = csv.writer(f)
       
    iterate = 0   
    ### run for 24 hours duration, once every few minutes, to gather enough 
    ### random tweet samples through out the day, different time zone around the world 
    while iterate < 720: 
#    while iterate < 2: 
        iterate = iterate + 1
        ### open csv file for writing: all the tweets return. 
#        files with timestamp every few minutes
#        filename_date_suffix = datetime.now().strftime("%Y%m%d_%H%M%S")
#        timestamp_f = open("TweetV3_" + filename_date_suffix + ".csv", "w", newline="")
#        tweet_writer2 = csv.writer(timestamp_f)
        
        ### query term hamradio, not just hash tags
        query_text = "hamradio OR amateurRadio"    
        kw = dict(q=query_text, count=100)
     
        for i in range(180):
        # Iterate through batches of results by following the cursor until we
        # reach the desired number of results, keeping in mind that OAuth users
        # can "only" make 180 search queries per 15-minute interval. See
        # https://dev.twitter.com/docs/rate-limiting/1.1/limits
            search_results = twitter_service_api.search.tweets(**kw)
            status_collection = search_results['statuses']           
            for status in status_collection: #100
                cur_tweet_id = status["id"]
                cur_user_id = status['user']['id']
                cur_created_at = status['created_at']
                cur_time_zone = status['user']['time_zone']
                cur_text = status['text'] 
                
                print ([cur_tweet_id, cur_user_id, cur_created_at, cur_time_zone, cur_text])
                tweet_writer.writerow([cur_tweet_id, cur_user_id, cur_created_at, cur_time_zone])
#                tweet_writer2.writerow([cur_tweet_id, cur_user_id, cur_created_at, cur_time_zone])
            try:
                next_results_metadata = search_results['search_metadata']['next_results']
            except KeyError as e: # No more results when next_results doesn't exist
                break

            kw = dict([ kv.split('=') for kv in next_results_metadata[1:].split("&") ])    
            # Create a dictionary from next_results, which has the following form:
            # ?max_id=313519052523986943&q=NCAA&include_entities=1        

        #timestamp_f.close()
        
        ### not returning all 180 requests sent per 15 minutes window, send request 
        ### more frequent than every 15 minutes, since never hit rate limit
        time.sleep(120) ### sleep for 2 minutes
        
    f.close()
    
######################################################################################################
main()
#quit()

[675415365766238208, 95508483, 'Fri Dec 11 20:42:26 +0000 2015', 'Madrid', 'HF2016HNY Happy New Year 2016  https://t.co/sNpqCwVhSq #hamradio #HNY #2016 https://t.co/wcEVj5PTdS']
[675414888655814656, 4341714095, 'Fri Dec 11 20:40:32 +0000 2015', 'Pacific Time (US & Canada)', 'RT @EA1WK: XR400AA 400 años Cabo de Hornos https://t.co/383AgfE2ma #hamr #hamradio']
[675414778139967488, 2600178026, 'Fri Dec 11 20:40:06 +0000 2015', 'Athens', 'KB6NU: Dollar stores provide rechargeable bargains https://t.co/tnk9xrYknv']
[675414515324985344, 3851416515, 'Fri Dec 11 20:39:03 +0000 2015', 'Pacific Time (US & Canada)', 'RT @HamRadioCoin: #HamRadioCoin Giveaway Christmas 2015!\nhttps://t.co/3fUs0weUhD\n\n#giveaway @cubes_2 #hamradio @yo3fvr #altcoin #coin https…']
[675413714963668993, 398704637, 'Fri Dec 11 20:35:52 +0000 2015', None, 'RT @K7AGE: THEREMIN’S BUG, he was an interesting fellow. #hamradio https://t.co/hzFP6xISxS']
[675413246342483968, 2811530484, 'Fri Dec 11 20:34:01 +0000 2015', 'Cairo'